# INFO371 Problem Set: Bayes-Theorem based Spam Filter

In this problem set you will use Bayes Theorem to categorize
emails from Ling-Spam corpus into spam and non-spam.  Using a single-word-based Bayes approach does not give good results, but this problem set serves as a preparatory
work for understanding the Naive Bayes approach.


## Ling-Spam emails

The corpus contains ~ 2700 emails from academic accounts talking
about conferences, deadlines, papers etc, and peppered with wonderful
offers of viagra, lottery millions and similar spam messages.  The
emails have been converted into a csv file that contains three variables:

* spam --> true or false, this email is spam
* files --> the original file name for this email (not needed in this HW).
* message --> the content of the email in a single line


## (5pt) Explore and clean the data

First, let's load data and take a closer look at it.

1. (2pt) Load the lingspam-emails.csv.bz2 dataset.  Browse a handful of emails, both spam and non-spam ones, to see what kind of text we are working with here.Hint: check out textwrap module to print long strings on multiple lines.
  
  
2. (3pt) Ensure the data is clean: remove all cases with missing spam and empty message field.  We do not care about the file names.

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf

# loading the dataset
spamEmail = pd.read_csv("/content/lingspam-emails.csv.bz2", sep="\t")
spamEmail.shape
spamEmail.info()
spamEmail.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2893 entries, 0 to 2892
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   spam     2893 non-null   bool  
 1   files    2893 non-null   object
 2   message  2893 non-null   object
dtypes: bool(1), object(2)
memory usage: 48.2+ KB


,spam,files,message
0,False,3-1msg1.txt,Subject: re : 2 . 882 s - > np np > date : su...
1,False,3-1msg2.txt,Subject: s - > np + np the discussion of s - ...
2,False,3-1msg3.txt,Subject: 2 . 882 s - > np np . . . for me it ...
3,False,3-375msg1.txt,"Subject: gent conference "" for the listserv ""..."
4,False,3-378msg1.txt,Subject: query : causatives in korean could a...


In [33]:
from textwrap import wrap
afew = spamEmail.sample(6)
for i in range(len(afew)):
    print("\n---\n", afew.spam.iloc[i])
    print("\n".join(wrap(afew.message.iloc[i])), "\n")


---
 False
Subject: aaai fall symoposium on formalizing context  formalizing
context aaai-95 fall symposium mit , cambridge , massachusetts
november 10-12 , 1995 call for papers description the notion of
context has played an important role in ai systems for many years .
however , formal logical explication of contexts remains an area of
research in which there are significant open issues . this symposium
will provide a forum for discussing formalizations of contexts ,
approaches to resolving open issues , and application areas for
context formalisms . the most ambitious goal of formalizing contexts
is to make automated reasoning systems which are never permanently
stuck with the concepts they use at a given time because they can
always transcend the context they are in . such a capability would
allow the designer of a reasoning system to include only such
phenomena as are required for the system 's immediate purpose ,
retaining the assurance that if a broader system is required later

In [34]:
spamEmail = spamEmail.drop(columns=['files'])
spamEmail = spamEmail.dropna()

## (15pt) Create Document-term matrix (DTM)

The first serious step is to create the document-term matrix (DTM).
This is simply numeric indicators for selected words: does this email
contain the word (1) or not (0).  But before we get there, we have to
decide the words.


1. (2pt) Choose 10+ words which might be good to distinguish between spam/non-spam.  Use these four: ''viagra'', ''deadline'', ''million'', and ''and''.  Choose more words yourself (you may want to return here and reconsider your choice later).


2. (10pt) Convert your messages into DTM.  We do not use the full 60k-words DTM here but only a baby-DTM of the 10 words you picked above. You may add the DTM columns to the original data frame, or keep those in a separate structure.

Creating the DTM involves finding whether the word is contained in the message for all emails in data. You can loop over emails and check each one individually, but pandas string methods make life much easier.  You will want to do case-insensitive matching, checking for both upper and lower case.  You may consider something like this:

```
for w in list_of_words:
    emails[w] = emails.message.str.lower().str.contains(w)
```

  Note: It is more intuitive to work with your data if you
  convert the logical values returned by contains to numbers.
  
  
  
3. (3pt) Split your work data (i.e. the DTM) and target (the spam indicator) into training and validation chunks (80/20 is a good split).

In [35]:
words = ['viagra', 'deadline', 'million', 'and', 'unsubscribe', 'urgent', 'cialis', 'loan', 'suspended', 'opt-out']

for w in words:
    spamEmail[w] = spamEmail.message.str.lower().str.contains(w).astype(int)


print(spamEmail.head())

    spam                                            message  viagra  deadline  \
0  False  Subject: re : 2 . 882 s - > np np  > date : su...       0         0   
1  False  Subject: s - > np + np  the discussion of s - ...       0         0   
2  False  Subject: 2 . 882 s - > np np  . . . for me it ...       0         0   
3  False  Subject: gent conference  " for the listserv "...       0         0   
4  False  Subject: query : causatives in korean  could a...       0         0   

   million  and  unsubscribe  urgent  cialis  loan  suspended  opt-out  
0        0    1            0       0       0     0          0        0  
1        0    0            0       0       0     0          0        0  
2        0    0            0       0       0     0          0        0  
3        0    1            0       0       0     0          0        0  
4        0    1            0       0       0     0          0        0  


In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = spamEmail[['viagra', 'deadline', 'million', 'and', 'unsubscribe', 'urgent', 'cialis', 'loan', 'suspended', 'opt-out']]
y = spamEmail['spam'].astype(int)

Xt, Xv, yt, yv = train_test_split(X, y, test_size=0.20)
Xt.shape, Xv.shape

((2314, 10), (579, 10))

## (80pt) Estimate and validate

Now you are ready with the preparatory work and it's time to
dive into the real thing.  Let's rehearse the Bayes theorem here
again.  We want to estimate the probability that an email is spam, given it
contains a certain word:

$Pr(category = S|w = 1) = \frac{Pr(w=1|category = S) * Pr(category=S)}{Pr(w=1)}$.


In order to compute this probability, we need to calculate some other
probabilities:

* $Pr(category=S)$ --> Probability of spam in data

* $Pr(category=NS)$ --> Probablility for non-spam in data

* $Pr(w=1)$ --> Probability the word is seen in messages

* $Pr(w=0)$ --> probability the word is not seen in messages

* $Pr(w=1|category = S)$ --> & probability the word is seen in messages that are spam

* $Pr(w=1|category = NS)$ --> probability the word is seen in messages that are not spam

....


but it turns out we are still not done with preparations. Namely, you need to compute
quite a few different probabilities below, including $Pr(category=S)$, $Pr(category=NS)$, $Pr(w=1)$, $Pr(w=0)$, $Pr(w=1|category = S)$, $Pr(w=0|category = S)$, $Pr(w=1|category = NS)$, $Pr(w=0|category = NS)$.


1. (2pt) Design a scheme for your variable names that describes these probabilities so that a) you understand what they mean; and b) the others (including your grader) will understand those! Hint: you may get some ideas from the [Python notes](https://faculty.washington.edu/otoomet/machinelearning-py/python.html#base-language) in Section 2.3, Base Language.

The first task is to compute these probabilities.
Use only training data for this task.


In [ ]:
SE # probability of spam email out of all emails
NSE # probability of non-spam emails out of all emails

P_YW # probability of yes word in all emails
P_NW # probability of no word in all emails

P_YW_YS # probability of word given it is a spam email
P_NW_YS # probability of no word given it is a spam email
P_YW_NS # probability of word given it is not a spam email
P_NW_NS # probability of no word given it is not a spam email


2. (4pt) Compute the priors, the unconditional probabilities for an email being spam and non-spam, $Pr(category=S)$ and $Pr(category=NS)$.  These probabilities are based on the spam variable alone, not on the text.

In [7]:
training_set = pd.concat([Xt, yt], axis=1)

SE = training_set['spam'].mean()
NSE = 1 - SE

print(SE)
print(NSE)

0.1598962834917891
0.8401037165082109


The next tasks involve computing the following probabilities for each
word out of the list of 10 you picked above,
I recommend to avoid unneccessary complexity and
just to write a loop over the words, compute the
answers, and print the word and the corresponding results there.  



3. (4pt) For each word $w$, compute the normalizers, $Pr(w=1)$ and $Pr(w=0)$.
  
  Hint: this is $Pr(million = 1) = 0.0484$.  But note this value
  (and the following hints) depends on your random training/validation split!
  

In [37]:
for i in words:
    P_YW = np.mean(training_set[i])
    P_NW = 1 - P_YW
    print(i + ": with word: " + str(P_YW) + " no word: " + str(P_NW))

viagra: with word: 0.000432152117545376 no word: 0.9995678478824547
deadline: with word: 0.1512532411408816 no word: 0.8487467588591184
million: with word: 0.04753673292999136 no word: 0.9524632670700086
and: with word: 0.9446845289541919 no word: 0.05531547104580814
unsubscribe: with word: 0.008210890233362144 no word: 0.9917891097666378
urgent: with word: 0.005185825410544511 no word: 0.9948141745894555
cialis: with word: 0.02895419187554019 no word: 0.9710458081244598
loan: with word: 0.016853932584269662 no word: 0.9831460674157303
suspended: with word: 0.00216076058772688 no word: 0.9978392394122731
opt-out: with word: 0.001728608470181504 no word: 0.9982713915298185


4. (7pt) For each word $w$, compute $Pr(w=1|category = S)$ and $Pr(w=1|category = NS)$.  These probabilities are based on both the spam-variable and on the DTM component that corresponds to the word $w$.
  
  Hint: $Pr(million = 1|category = S) = 0.252$

In [39]:
for i in words:
    P_YW_YS = training_set[(training_set[i]==1) & (training_set['spam']==1)].shape[0] / training_set['spam'].sum()
    P_NW_YS = 1 - P_YW_YS
    P_YW_NS =  training_set[(training_set[i]==1) & (training_set['spam']==0)].shape[0] / ( training_set.shape[0] - training_set[i].sum())
    P_NW_NS = 1 - P_YW_NS

    print(i)
    print("P_YW_YS: "+ str(P_YW_YS))
    print("P_NW_YS: "+str(P_NW_YS))
    print("P_YW_NS: "+str(P_YW_NS))
    print("P_NW_NS: "+str(P_NW_NS))

viagra
P_YW_YS: 0.002702702702702703
P_NW_YS: 0.9972972972972973
P_YW_NS: 0.0
P_NW_NS: 1.0
deadline
P_YW_YS: 0.0
P_NW_YS: 1.0
P_YW_NS: 0.17820773930753564
P_NW_NS: 0.8217922606924644
million
P_YW_YS: 0.24594594594594596
P_NW_YS: 0.754054054054054
P_YW_NS: 0.008620689655172414
P_NW_NS: 0.9913793103448276
and
P_YW_YS: 0.9216216216216216
P_NW_YS: 0.07837837837837835
P_YW_NS: 14.4140625
P_NW_NS: -13.4140625
unsubscribe
P_YW_YS: 0.04594594594594595
P_NW_YS: 0.9540540540540541
P_YW_NS: 0.0008714596949891067
P_NW_NS: 0.9991285403050109
urgent
P_YW_YS: 0.008108108108108109
P_NW_YS: 0.9918918918918919
P_YW_NS: 0.003909643788010426
P_NW_NS: 0.9960903562119896
cialis
P_YW_YS: 0.010810810810810811
P_NW_YS: 0.9891891891891892
P_YW_NS: 0.028037383177570093
P_NW_NS: 0.9719626168224299
loan
P_YW_YS: 0.04864864864864865
P_NW_YS: 0.9513513513513514
P_YW_NS: 0.009230769230769232
P_NW_NS: 0.9907692307692307
suspended
P_YW_YS: 0.010810810810810811
P_NW_YS: 0.9891891891891892
P_YW_NS: 0.00043308791684711995

5. (5pt) Finally, compute the probabilities of interest, $Pr(category = S|w = 1)$ and $Pr(category = S|w = 0)$.  Compute this value using Bayes theorem, not directly by counting!
  
  For the check, you may also compute
  $Pr(category = NS|w = 1)$ and $Pr(category = NS|w = 0)$
  
  Hint: $\Pr(\mathit{category} = S|\mathit{million} = 1) = 0.843$.  But
  note this number depends on your random testing-validation split!


In [28]:
for i in words:
    PW = training_set[i].sum() / training_set.shape[0]
    P_NW = 1 - PW
    P_YW_YS = training_set[(training_set[i]==1) & (training_set['spam']==1)].shape[0] / training_set['spam'].sum()
    P_NW_YS = 1 - P_YW_YS
    prYSw1 = (P_YW_YS * SE)/PW
    prYSw0 = (P_NW_YS * SE)/P_NW
    print(i+":")
    print("probability of email being spam if word is present: "+str(prYSw1))
    print("probability of email being spam if word is not present: "+str(prYSw0))

viagra:
probability of email being spam if word is present: 1.0
probability of email being spam if word is not present: 0.15953307392996108
deadline:
probability of email being spam if word is present: 0.0
probability of email being spam if word is not present: 0.18839103869653767
million:
probability of email being spam if word is present: 0.8272727272727272
probability of email being spam if word is not present: 0.1265880217785844
and:
probability of email being spam if word is present: 0.15599268069533395
probability of email being spam if word is not present: 0.22656249999999986
unsubscribe:
probability of email being spam if word is present: 0.8947368421052632
probability of email being spam if word is not present: 0.15381263616557733
urgent:
probability of email being spam if word is present: 0.25000000000000006
probability of email being spam if word is not present: 0.15942658557775843
cialis:
probability of email being spam if word is present: 0.05970149253731344
probability of

6. (6pt)  Which of these probabilities have to sum to one? (E.g. $Pr(category = 1) + Pr(category = 0) = 1$.) Which ones do not?  Explain!

In [ ]:
print(P_YS_YW + P_NS_YW) # (Pr(S=1) | w = 1 )) + (Pr(S = 0) | w = 1 ))
print(P_YS_NW + P_NS_NW) # (Pr(S = 1) | w = 0 )) + (Pr(S = 0) | w = 0 ))
print(P_YW_YS + P_NW_YS) # (Pr(w = 1) | S = 1) + (Pr(w = 0) | S = 1)
print(P_YW_YN + P_NW_YN) # (Pr(w = 1) | S = 0) + (Pr(w = 0) | S = 0)

#The reason behind why these probabilities sum to one is because they have a common denominator (normalizer)

Now we are done with the estimator.  Your fitted model is completely
described by these probabilities.  Let's now turn to prediction, using
your validation data.  Note that we are still inside the loop over
each word $w$!

9. (8pt) For each email in your validation set, predict whether it is predicted to be spam or non-spam.  Hint: you should check if it contains the word $w$ and use the appropriate probability, $Pr(category = S|w = 1)$ or $Pr(category = S|w = 0)$.

10. (5pt) Print the resulting confusion matrix and compute accuracy, precision and recall.

In [40]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

valid_set = pd.concat([Xv, yv], axis=1)
for i in words:
    PW = training_set[i].sum() / training_set.shape[0]
    P_NW = 1 - PW
    P_YW_YS = training_set[(training_set[i]==1) & (training_set['spam']==1)].shape[0] / training_set['spam'].sum()
    P_NW_YS = 1 - P_YW_YS
    prYSw1 = (P_YW_YS * SE)/PW
    prYSw0 = (P_NW_YS * SE)/P_NW
    print(i+":")
    print(str(prYSw1))
    print(str(prYSw0))

    P_S_W = np.where(valid_set[i], prYSw1, prYSw0)
    predictedSpam = (P_S_W > 0.5) + 0

    #cm = confusion_matrix(valid_set['spam'], predictedSpam)
    accuracy = accuracy_score(valid_set['spam'], predictedSpam)
    recall = recall_score(valid_set['spam'], predictedSpam)
    precision = precision_score(valid_set['spam'], predictedSpam)

    #print("cm: "+ str(cm))
    print("precision: "+ str(precision))
    print("accuracy: "+ str(accuracy))
    print("recall:" + str(recall))

viagra:
1.3127027027027027
0.20941949731833
precision: 1.0
accuracy: 0.8393782383419689
recall:0.010638297872340425
deadline:
0.0
0.24730142566191446
precision: 0.0
accuracy: 0.8376511226252159
recall:0.0
million:
1.085963144963145
0.16617243831853631
precision: 0.7037037037037037
accuracy: 0.8566493955094991
recall:0.20212765957446807
and:
0.2047720135506046
0.29740920608108096
precision: 0.0
accuracy: 0.8376511226252159
recall:0.0
unsubscribe:
1.1745234708392605
0.20191026320438088
precision: 1.0
accuracy: 0.846286701208981
recall:0.05319148936170213
urgent:
0.32817567567567574
0.20927970977058727
precision: 0.0
accuracy: 0.8376511226252159
recall:0.0
cialis:
0.07837031060911659
0.2138180637246058
precision: 0.0
accuracy: 0.8376511226252159
recall:0.0
loan:
0.605862785862786
0.20310828630828634
precision: 0.14285714285714285
accuracy: 0.8290155440414507
recall:0.010638297872340425
suspended:
1.0501621621621622
0.2080767384968338
precision: 0.0
accuracy: 0.8376511226252159
recall:0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

11. (5pt) Which steps above constitute model training?  In which steps do you use trained model?  What is a trained model in this case? Explain!
  
  Hint: a trained model is all you need to make predictions.

*the steps that consititute as model training would be calculating the naive bayes probabilities. In this assignment model training would constitute from q1 - q6. q9 and 10 would be utilizing the trained model to predict if an email was spam or not spam in the validation set.*

---
Now it is time to look at your results a little bit closer.

12. (4pt) Comment the overall performance of the model--how do accuracy, precision and recall look like?


13. (8pt) Explain why do you see very low recall while the other indicators do not look that bad.


14. (8pt) Explain why some words work well and others not:
  * why does ''million'' improve accuracy?
  * why does ''viagra'' not work?
  * why does ''deadline'' not work?
  * why does ''and'' not work?

  Hint: You may just see where in which emails these words occur, and
  how frequently.  These are all different reasons!

*Answer 12. The accuracies for the words are mostly in 80 - 85 range indicating the model overall predicts if an email is spam or not spam pretty well. However the percision and recall are very low. This means the model doesn't do well in terms determining percision meaning there might be a lot of false positives. In addition recall is low indicating there's missing instances that aren't showing up in the results.*

*Answer 13. The low recall may be due higher false negatives. It may be because the outcome is of an imbalanced class or the model hyperparameters are untuned. In order to improve it we can implement class weights.*

*Answer 14. Million based on the prior probabilities, shows up in almost half of spam emails. Viagra doesn't work because it only appears in one email. Deadline doesn't show up in spam emails despite showing up in multiple emails. And shows up in many emails however the ratio of it showing up in spam emails is small, therefore not really indicating much.*

In [47]:
print("million:" +str(training_set['million'].sum()) +" In spam emails: "+str(training_set[(training_set['million']==1) & (training_set['spam']==1)].shape[0]))
print("viagra:" +str(training_set['viagra'].sum()) + " In spam emails: "+str(training_set[(training_set['viagra']==1) & (training_set['spam']==1)].shape[0]))
print("deadline:" +str(training_set['deadline'].sum()) + " In spam emails: "+str(training_set[(training_set['deadline']==1) & (training_set['spam']==1)].shape[0]))
print("and:" +str(training_set['and'].sum()) + " In spam emails: "+str(training_set[(training_set['and']==1) & (training_set['spam']==1)].shape[0]))

million:110 In spam emails: 91
viagra:1 In spam emails: 1
deadline:350 In spam emails: 0
and:2186 In spam emails: 341



15. (5pt) Add such smoothing to the model.  You can either literally add two such lines of data, or alternatively manipulate the way you compute the probabilities.


16. (5pt) Repeat the tasks above: compute the probabilities, do predictions, compute the accuracy, precision, recall for all words.  

In [48]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

valid_set = pd.concat([Xv, yv], axis=1)
for i in words:
    alpha = 0.05

    SE = (training_set['spam'].mean()) + alpha
    NSE = (1 - SE) + alpha

    PW = training_set[i].sum() / training_set.shape[0]
    P_NW = 1 - PW

    P_YW_YS = ((training_set[(training_set[i]==1) & (training_set['spam']==1)].shape[0])+alpha) / ((training_set['spam'].sum())*alpha)
    P_NW_YS = 1 - P_YW_YS
    prYSw1 = (P_YW_YS * SE)/PW
    prYSw0 = (P_NW_YS * SE)/P_NW
    print(i+":")
    print(str(prYSw1))
    print(str(prYSw0))

    P_S_W = np.where(valid_set[i], prYSw1, prYSw0)
    predictedSpam = (P_S_W > 0.5) + 0

    cm = confusion_matrix(valid_set['spam'], predictedSpam)
    accuracy = accuracy_score(valid_set['spam'], predictedSpam)
    recall = recall_score(valid_set['spam'], predictedSpam)
    precision = precision_score(valid_set['spam'], predictedSpam)

    #print("cm: "+ str(cm))
    print("precision: "+ str(precision))
    print("accuracy: "+ str(accuracy))
    print("recall:" + str(recall))

viagra:
27.56675675675676
0.19806884705717392
precision: 1.0
accuracy: 0.8393782383419689
recall:0.010638297872340425
deadline:
0.0037505791505791507
0.24663304343039577
precision: 0.0
accuracy: 0.8376511226252159
recall:0.0
million:
21.73119656019656
-0.8642157992838575
precision: 0.7037037037037037
accuracy: 0.8566493955094991
recall:0.20212765957446807
and:
4.096040775450657
-66.15816511824323
precision: 0.15441176470588236
accuracy: 0.18825561312607944
recall:0.8936170212765957
unsubscribe:
23.559559032716926
0.016587528705175762
precision: 1.0
accuracy: 0.846286701208981
recall:0.05319148936170213
urgent:
6.672905405405405
0.17620553220466342
precision: 0.4
accuracy: 0.8359240069084629
recall:0.02127659574468085
cialis:
1.5869987898346107
0.16883448201205212
precision: 0.047619047619047616
accuracy: 0.8048359240069085
recall:0.010638297872340425
loan:
12.150914760914763
0.005193109593109588
precision: 0.14285714285714285
accuracy: 0.8290155440414507
recall:0.010638297872340425
sus

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


17. (4pt) Comment on the results.  Does smoothing improve the overall performance?

*Based on results we can see the results for accuracy is pretty similar to the previous model. In addition the percision and accuracy has gone higher ( especially for "and" ). However we can also see the naive bayes probabilities is pretty extreme for words like "and" and viagra. This suggests there's some overconfidence in the model.*